In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType


In [45]:
spark.stop()

In [33]:
spark = SparkSession.builder.master("local").appName("RandomForestClassifierExample").getOrCreate()


In [34]:
exp = {"id": "JPY=X", "exchange": "CCY", "quoteType": 14, "price": 143.16700744628906, "timestamp": 1745361575000, "marketHours": 1, "changePercent": 1.6818382740020752, "dayVolume": 0, "change": 2.368011474609375, "priceHint": 4}

In [35]:
import pandas as pd 
df_pandas = pd.DataFrame(exp, index=[0])

In [36]:
df_pandas.head()


,id,exchange,quoteType,price,timestamp,marketHours,changePercent,dayVolume,change,priceHint
0,JPY=X,CCY,14,143.167007,1745361575000,1,1.681838,0,2.368011,4


In [37]:
df= spark.createDataFrame(df_pandas)

In [38]:
df.show()

+-----+--------+---------+------------------+-------------+-----------+------------------+---------+-----------------+---------+
|   id|exchange|quoteType|             price|    timestamp|marketHours|     changePercent|dayVolume|           change|priceHint|
+-----+--------+---------+------------------+-------------+-----------+------------------+---------+-----------------+---------+
|JPY=X|     CCY|       14|143.16700744628906|1745361575000|          1|1.6818382740020752|        0|2.368011474609375|        4|
+-----+--------+---------+------------------+-------------+-----------+------------------+---------+-----------------+---------+



In [40]:
df.show()

+-----+--------+---------+------------------+-------------+-----------+------------------+---------+-----------------+---------+
|   id|exchange|quoteType|             price|    timestamp|marketHours|     changePercent|dayVolume|           change|priceHint|
+-----+--------+---------+------------------+-------------+-----------+------------------+---------+-----------------+---------+
|JPY=X|     CCY|       14|143.16700744628906|1745361575000|          1|1.6818382740020752|        0|2.368011474609375|        4|
+-----+--------+---------+------------------+-------------+-----------+------------------+---------+-----------------+---------+



In [ ]:

# df = df.withColumn("timestamp_corrected", to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss")).drop(col('timestamp'))



+-----+--------+---------+------------------+-----------+------------------+---------+-----------------+---------+-------------------+-------------+
|   id|exchange|quoteType|             price|marketHours|     changePercent|dayVolume|           change|priceHint|timestamp_corrected|previos_value|
+-----+--------+---------+------------------+-----------+------------------+---------+-----------------+---------+-------------------+-------------+
|JPY=X|     CCY|       14|143.16700744628906|          1|1.6818382740020752|        0|2.368011474609375|        4|2025-04-22 15:39:35|          0.0|
+-----+--------+---------+------------------+-----------+------------------+---------+-----------------+---------+-------------------+-------------+



In [42]:
df = df.withColumn("timestamp_corrected", to_timestamp((col('timestamp')/1000).cast("timestamp"), "yyyy-MM-dd HH:mm:ss")).drop(col('timestamp'))
df.show()

+-----+--------+---------+------------------+-----------+------------------+---------+-----------------+---------+-------------------+
|   id|exchange|quoteType|             price|marketHours|     changePercent|dayVolume|           change|priceHint|timestamp_corrected|
+-----+--------+---------+------------------+-----------+------------------+---------+-----------------+---------+-------------------+
|JPY=X|     CCY|       14|143.16700744628906|          1|1.6818382740020752|        0|2.368011474609375|        4|2025-04-22 15:39:35|
+-----+--------+---------+------------------+-----------+------------------+---------+-----------------+---------+-------------------+



In [ ]:
partition = Window.partitionBy(col('id')).orderBy(col('timestamp_corrected'))
df = df.withColumn("previos_value", lag("price", 1, 0).over(partition))